In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = os.getenv('MLFLOW_TRACKING_URI')
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_TRACKING_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_TRACKING_PASSWORD")

In [3]:
%pwd

'c:\\Users\\Smit\\Desktop\\DESKTOP\\Github Repos\\DS_project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Smit\\Desktop\\DESKTOP\\Github Repos\\DS_project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path 
    test_path: Path
    model_path: Path 
    metrics_path: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [8]:
from src.DS_project.constants import *
from src.DS_project.utils.common import *

In [9]:
class ConfigurationManager:
    def __init__(self , 
                 config_path =CONFIG_FILE_PATH , 
                 params_path =PARAMS_FILE_PATH , 
                 schema_path=SCHEMA_FILE_PATH):
        self.config_path = read_yaml(config_path)
        self.params_path = read_yaml(params_path)
        self.schema_path = read_yaml(schema_path)

        create_directories([self.config_path.artifacts_root])

    def get_model_evaluation(self) -> ModelEvaluationConfig:
        config = self.config_path.model_evaluation
        create_directories([config.root_dir])
        schema = self.schema_path.TARGET_COLUMN
        params = self.params_path.ElasticNet

        model_evaluation = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_path=config.test_path,
            model_path=config.model_path,
            metrics_path=config.metrics_path,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/smitngandhi/DS_project.mlflow"
        )


        return model_evaluation

In [10]:
from sklearn.metrics import root_mean_squared_error , mean_absolute_error , r2_score
import mlflow
import pandas as pd
from urllib.parse import urlparse


In [13]:
class Model_evaluation:
    def __init__(self , config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual , predicted):
        rmse = root_mean_squared_error(actual , predicted)
        mae = mean_absolute_error(actual , predicted)
        r2 = r2_score(actual , predicted)

        return rmse , mae , r2
    
    def evaluate_with_mlflow(self):

        test = pd.read_csv(self.config.test_path)

        test_x = test.drop(self.config.target_column , axis = 1)
        test_y = test[self.config.target_column]

        model = joblib.load(self.config.model_path)
        logger.info("Model is loaded successfully")

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predictions = model.predict(test_x)

            logger.info("Predictions are ready")

            (rmse , mae , r2) = self.eval_metrics(test_y , predictions)

            scores = {'rmse': rmse , 'mae': mae , 'r2': r2}

            save_json(path=(Path(self.config.metrics_path)) , data=scores)

            mlflow.log_params(self.config.all_params)

            print("Tracking URI:", mlflow.get_tracking_uri())

            mlflow.log_metric('rmse' , rmse)
            mlflow.log_metric('mae', mae)
            mlflow.log_metric('r2' , r2)

            # mlflow.sklearn.log_model(sk_model=model)

            

In [14]:
try:
    cm = ConfigurationManager()
    model_evaluation_config = cm.get_model_evaluation()
    model_evaluation = Model_evaluation(config=model_evaluation_config)
    model_evaluation.evaluate_with_mlflow()
except Exception as e:
    logger.exception(e)
    raise e

2025-07-03 15:32:38,435: INFO: common: Yaml file: config\config.yaml is loaded successfully
2025-07-03 15:32:38,437: INFO: common: Yaml file: params.yaml is loaded successfully
2025-07-03 15:32:38,439: INFO: common: Yaml file: schema.yaml is loaded successfully
2025-07-03 15:32:38,441: INFO: common: Directory artifacts is created
2025-07-03 15:32:38,441: INFO: common: Directory artifacts/model_evaluation is created
2025-07-03 15:32:38,448: INFO: 1375258951: Model is loaded successfully
2025-07-03 15:32:38,842: INFO: 1375258951: Predictions are ready
2025-07-03 15:32:38,848: INFO: common: json file saved at artifacts\model_evaluation\metrics.json
Tracking URI: https://dagshub.com/smitngandhi/DS_project.mlflow
🏃 View run vaunted-frog-379 at: https://dagshub.com/smitngandhi/DS_project.mlflow/#/experiments/0/runs/91026d07001d4f49a86b1265a756dad6
🧪 View experiment at: https://dagshub.com/smitngandhi/DS_project.mlflow/#/experiments/0
